# K-Vizinhos Mais Próximos Classificador

## Introdução

Este notebook visa fazer a otimização de **hiperparâmetros**, de estratégia de **normalização**, de **seleção de atributos** e de **redução de dimensionalidade**. A seleção de atributos é realizada por *Sequential Feature Selection* (**SFS**, ou Seleção Sequencial de Atributos em português) por remoção de atributo, a otimização testa a ausência ou não dessa estratégia. A redução de dimensionalidade é realizada por *Principal Component Analysis* (**PCA**, ou Análise de Componenetes Principais em português), é testada sua ausência também. Para normalização, são testadas a ausência, ou a normalização **padrão** ou a pelos **mínimos e máximos**. A otimização é feita usando o módulo **optuna**.

### Modelo

O modelo otimizado é um K-nn classificador do scikit-learn [2]. Os hiperparâmetros otimizados são:

- `n_neighbors`: número de vizinhos. Testados no **intervalo de 2 até 80**, que é um pouco mais de 10% dos 782 registros de tsunami;
    
- `weights`: pesos no cálculo da moda entre os vizinhos mais próximos. Testado entre **'uniform'**, que dá o mesmo peso para todos os vizinhos, e **'distance'**, que dá mais peso aos vizinhos mais próximos;
    
- `p`: potência minkowski. Testado no **intervalo entre 1 e 2**, sendo 1 a distância Manhattan e 2 a distância Euclidiana.

## Carrega os dados

In [1]:
import pandas as pd

# Dados de treino
X_treino = pd.read_csv('../Dados/dados_tratados/X_treino.csv')
y_treino = pd.read_csv('../Dados/dados_tratados/y_treino.csv').values.ravel()

## Define algumas variáveis úteis

In [2]:
from sklearn.metrics import fbeta_score, make_scorer

SEMENTE_ALEATORIA = 9 # Random state

f2_score = make_scorer(fbeta_score, beta=2) # Métrica de avaliação de desempenho
num_max_atributos = len(X_treino.columns) - 1 # Para SFS

X_treino = X_treino.values

## Definição de funções para o optuna

O código abaixo define a função que cria uma instância do modelo. É nela que o pipeline é montado de acordo com as sugestões definidas pelo optuna.

In [3]:
from sklearn.neighbors import KNeighborsClassifier # algoritmo de aprendizado de máquina
from sklearn.preprocessing import StandardScaler, MinMaxScaler # normalização
from sklearn.feature_selection import SequentialFeatureSelector # seleção de atributos
from sklearn.decomposition import PCA # redução de dimensionalidade
from sklearn.pipeline import make_pipeline

def cria_instancia_modelo(trial):
    """Cria uma instância do modelo.

    Args:
      trial: objeto tipo Trial do optuna.

    Returns:
      Uma instância do modelo desejado.

    """

    # trial.suggest_ do optuna
    
    # Hiperparâmetros do algoritmo
    parametros = {
        "n_neighbors": trial.suggest_int("vizinhos", 2, 80),
        "weights": trial.suggest_categorical("pesos", ["uniform", "distance"]), 
        "p": trial.suggest_float("p_minkowski", 1, 2), 
        "n_jobs": 1,
    }

    # Estratégias de tratamento de dados
    normalizacao = trial.suggest_categorical("normalizacao", ["none", "standard", "minmax"])
    usar_sfs = trial.suggest_categorical("SFS", [True, False])
    usar_pca = trial.suggest_categorical("PCA", [True, False])

    # Hiperparâmetros de SFS e PCA, respectivamente
    num_atributos = trial.suggest_int("num_atributos", 2, num_max_atributos) # para SFS
    num_dimensoes = trial.suggest_int("num_dimensoes", 2, num_atributos) # para PCA, depende do num_atributos do SFS
        
    # Monta o pipeline    
    
    steps = []

    # Normalização
    if normalizacao == "standard":
        steps.append(("normalizador", StandardScaler()))
    elif normalizacao == "minmax":
        steps.append(("normalizador", MinMaxScaler()))

    # SFS
    if usar_sfs:
        steps.append(("sfs", SequentialFeatureSelector(
            KNeighborsClassifier(**parametros), 
            n_features_to_select=num_atributos,
            direction="backward",
            scoring=f2_score,
            cv=5,
            n_jobs=1,
        )))

    # PCA
    if usar_pca:
        steps.append(("pca", PCA(n_components=num_dimensoes, random_state=SEMENTE_ALEATORIA)))

    steps.append(("modelo", KNeighborsClassifier(**parametros)))


    # Faz pipeline
    modelo = make_pipeline(*[s[1] for s in steps])

    return modelo

O código abaixo define a função que estima o desempenho do modelo de cada trial. Essa é a função que terá valor de retorno maximizado pelo optuna.

In [4]:
from sklearn.model_selection import cross_val_score
import numpy as np

def funcao_objetivo(trial, X, y, num_folds):
    """Função objetivo do optuna, função cujo retorno será maximizado.

    Faz validação cruzada estratificada com métrica F2.

    Args:
        - trial: objeto tipo Trial do optuna;
        - x: ndarray. dataset de treino com atributos;
        - y: ndaray. dataset de treino com target;
        - num_folds: int. Número de folds da validação cruzada.
    
    Returns:
        - média dos resultados da validação cruzada.
    """
    modelo = cria_instancia_modelo(trial)

    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring=f2_score,
        cv=num_folds,
    )

    return np.mean(metricas)

## Otimização com optuna

O código abaixo cria ou carrega um 'estudo' a depender da existência dele. O 'estudo' contém informações sobre cada trial. A variável objeto_de_estudo acessa esse 'estudo'. 

In [2]:
from optuna import create_study

NOME_DO_ESTUDO = "Knn_class_optuna"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-10-31 20:07:14,494] Using an existing study with name 'Knn_class_optuna' instead of creating a new one.


O código abaixo faz a otimização com optuna.

In [6]:
# Parâmetros
NUM_FOLDS = 10 # Número de folds da validação cruzada
NUM_TENTATIVAS = 200 # Quantidade de trials

def funcao_objetivo_parcial(trial):
    '''Função objetivo no formato exigido pelo optuna.'''
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

# Faz a otimização
objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2025-10-30 18:54:56,331] Trial 1 finished with value: 0.8190777986823861 and parameters: {'vizinhos': 13, 'pesos': 'distance', 'p_minkowski': 1.5840645960411315, 'normalizacao': 'standard', 'SFS': True, 'PCA': False, 'num_atributos': 7, 'num_dimensoes': 4}. Best is trial 1 with value: 0.8190777986823861.
[I 2025-10-30 18:54:57,124] Trial 2 finished with value: 0.8520135673325988 and parameters: {'vizinhos': 61, 'pesos': 'distance', 'p_minkowski': 1.7239051033127128, 'normalizacao': 'minmax', 'SFS': False, 'PCA': False, 'num_atributos': 7, 'num_dimensoes': 3}. Best is trial 2 with value: 0.8520135673325988.
[I 2025-10-30 18:55:10,671] Trial 3 finished with value: 0.67679520861162 and parameters: {'vizinhos': 68, 'pesos': 'distance', 'p_minkowski': 1.2659266615051732, 'normalizacao': 'standard', 'SFS': True, 'PCA': True, 'num_atributos': 7, 'num_dimensoes': 3}. Best is trial 2 with value: 0.8520135673325988.
[I 2025-10-30 18:55:12,228] Trial 4 finished with value: 0.7768684262306248 a

## DataFrame dos trials

O código abaixo mostra o DataFrame contruído pelo objeto de estudo do optuna contendo todos os parâmetros de cada trial.

In [3]:
df_analysis = objeto_de_estudo.trials_dataframe()

df_analysis

,number,value,datetime_start,datetime_complete,duration,params_PCA,params_SFS,params_normalizacao,params_num_atributos,params_num_dimensoes,params_p_minkowski,params_pesos,params_vizinhos,state
0,0,NaN,2025-10-30 18:54:38.615988,NaT,NaT,True,False,standard,3,2,1.067813,uniform,64,RUNNING
1,1,0.819078,2025-10-30 18:54:46.774941,2025-10-30 18:54:56.133398,0 days 00:00:09.358457,False,True,standard,7,4,1.584065,distance,13,COMPLETE
2,2,0.852014,2025-10-30 18:54:56.342576,2025-10-30 18:54:57.037469,0 days 00:00:00.694893,False,False,minmax,7,3,1.723905,distance,61,COMPLETE
3,3,0.676795,2025-10-30 18:54:57.134654,2025-10-30 18:55:09.765389,0 days 00:00:12.630735,True,True,standard,7,3,1.265927,distance,68,COMPLETE
4,4,0.776868,2025-10-30 18:55:10.683625,2025-10-30 18:55:12.115370,0 days 00:00:01.431745,False,False,none,8,4,1.833747,uniform,26,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,196,0.886788,2025-10-30 19:37:36.047430,2025-10-30 19:37:48.172381,0 days 00:00:12.124951,True,True,none,5,3,1.087314,uniform,35,COMPLETE
197,197,0.883636,2025-10-30 19:37:48.278425,2025-10-30 19:37:59.576206,0 days 00:00:11.297781,True,True,none,5,3,1.079747,uniform,34,COMPLETE
198,198,0.874643,2025-10-30 19:37:59.911588,2025-10-30 19:38:09.244795,0 days 00:00:09.333207,True,True,none,6,3,1.099409,uniform,31,COMPLETE
199,199,0.886788,2025-10-30 19:38:09.445795,2025-10-30 19:38:20.662421,0 days 00:00:11.216626,True,True,none,5,3,1.080786,uniform,35,COMPLETE


## Melhor modelo

O código abaixo mostra o melhor modelo encontrado pelo optuna e seus parâmetros. Note que os parâmetros 'num_atributos' e 'num_dimensoes' só influenciam caso 'SFS' e 'PCA' sejam True respectivamente, pois são seus respectivos hiperparâmetros.

In [8]:
melhor_trial = objeto_de_estudo.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Número do melhor trial: 75
Parâmetros do melhor trial: {'vizinhos': 38, 'pesos': 'uniform', 'p_minkowski': 1.1213453726046692, 'normalizacao': 'minmax', 'SFS': True, 'PCA': False, 'num_atributos': 3, 'num_dimensoes': 3}


Este notebook se encerra aqui. O modelo será usado no notebook chamado 'Resultados_Discussão'.

## Referências

[1] Documentação do optuna. Acesso em: 31/10/2025. Disponível em: https://optuna.readthedocs.io/en/stable/ 

[2] Página da Regressão Logística da documentação do scikit-learn. Acesso em: 31/10/2025. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

[3] Sperat, Walter. *Using optuna with sklearn the right way*. Acesso em 31/10/2025. Disponível em: https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451